In [1]:
import os
import math
import pandas as pd

In [2]:
data_path = os.getcwd() + "/dataset/original_data"
first_processing_path = os.getcwd() +"/dataset/raw_labelled"
second_processing_path = os.getcwd() +"/dataset/windows"
final_dataset_path = os.getcwd() +"/dataset"
test_patients = ["S01", "S02"]

window_length = 3*64

In [3]:
'''
people[] -> 存所有在資料夾中的txt檔

'''
people = [] #S01R01.txt~S10R01.txt

for person in os.listdir(data_path):
    if '.txt' in person:
        people.append(person)


del person

In [4]:
def label_prefog(dataset, window_length):
# def label_prefog(dataset, window_length = 1):
    # Drop all the rows for which Action is 0 or the rows which are not part of the experiment
    # 將所有的Action==0(未實驗)的部分刪除，並不回傳任何值(implace的部分)
    dataset.drop(index = list(dataset[dataset['Action'] == 0].index),
                 inplace = True)

    time = dataset["time"].values
    dataset.drop(["time"], axis=1, inplace=True)
    time = (time-time.min())/1000
    temp = pd.Series(time)
    dataset["time"] = temp.values
    dataset = dataset[['time',
                                'A_F', 'A_V', 'A_L',
                                'L_F', 'L_V', 'L_L',
                                'T_F', 'T_V', 'T_L',
                                'Action']]
    # window_length = int(64*window_length)
    #print(window_length)
    fog_index = []
    for i in dataset.index:
        if dataset.loc[i, 'Action'] == 2: # 取index和'Action'對應到的資料
            fog_index.append(i) # 將發生fog的index存入fog_index這個array
    fog_index
    
    # print(fog_index)
    
    start_indices = []
    for i in fog_index:
        if (dataset.loc[i-1, 'Action'] != dataset.loc[i, 'Action']):
            start_indices.append(i) # 將發生fog之前的index存入start_indices這個array
    
    # print(start_indices)

    prefog = []
    for start in start_indices:
        prefog_start = [x for x in range(start-window_length, start)] # prefog_start把FOG之前的windows標記起來
        prefog.append(prefog_start)
        
    # print(prefog) # 2D array

    prefog = [item for sublist in prefog for item in sublist] # 把[][]轉換成[]

    # print(prefog) # 1D array

    for i in prefog:
        if dataset.loc[i, "Action"] == 2:
            print (i)
        dataset.loc[i,'Action'] = 3 # 把紀錄在Pre-fog的index action 記錄成3
    dataset['Action'] = dataset['Action'] - 1 # 所有action編號-1 使0=walk 1=fog 2= pre-fog
    return dataset

In [5]:
dataset = pd.DataFrame()
last_name = '' # 紀錄哪個是最終的病患
patients = [] # 病患ID list
i = 1
for person in people:
    name = person.split('R')[0]

    if name != last_name:
        dataset = dataset.drop(index=dataset.index)
        patients.append(name)
    last_name = name
    print(name)

    file = data_path + "\\" + person

    if name in person:
        temp = pd.read_csv(file, delimiter = " ", header = None)
        print(person, ' is read', end = "\t")
        
        if 2 in temp[max(temp.columns)].unique():
            print("Adding {} to dataset".format(person), end = "\t")
            temp.columns = [
                                'time',
                                'A_F', 'A_V', 'A_L',
                                'L_F', 'L_V', 'L_L',
                                'T_F', 'T_V', 'T_L',
                                'Action'                           
                                ]
            
            temp = label_prefog(temp, window_length).reset_index(drop = True)
            temp['name'] = name
            print("{} is labelled".format(person))
            dataset = pd.concat([dataset, temp],axis = 0)
        print('')

        dataset.reset_index(drop = True, inplace = True)
        
        to_name = first_processing_path + "/win_" + name + ".csv"
    # 存入/raw_labelled中

    if not dataset.empty:
        dataset.to_csv(to_name, index = False)

del last_name, name, person, people, dataset, temp, to_name


S01
S01R01.txt  is read	Adding S01R01.txt to dataset	S01R01.txt is labelled

S01
S01R02.txt  is read	Adding S01R02.txt to dataset	34337
34338
34339
34340
34341
34342
34343
34344
34345
34346
34347
34348
34349
34350
34351
34352
34353
34354
34355
34356
34357
34358
34359
34360
34361
34362
34363
34364
34365
34366
34367
34368
34369
34370
34371
34372
34373
34374
34375
34376
34377
34378
34379
34380
34381
34382
34383
34384
S01R02.txt is labelled

S02
S02R01.txt  is read	Adding S02R01.txt to dataset	56081
56082
56083
56084
56085
56086
56087
56088
56089
56090
56091
56092
56093
56094
56095
56096
56097
56098
56099
56100
56101
56102
56103
56104
56105
56106
56107
56108
56109
56110
56111
56112
56113
56114
56115
56116
56117
56118
56119
56120
56121
56122
56123
56124
56125
56126
56127
56128
56129
56130
56131
56132
56133
56134
56135
56136
56137
56138
56139
56140
56141
56142
57713
57714
57715
57716
57717
57718
57719
57720
57721
57722
57723
57724
57725
57726
57727
57728
57729
57730
57731
57732
57733
57734
5

In [6]:
def create_window(act,window_length,dataframe):

  indices = list(dataframe[dataframe.Action == act].index)
  time = []
  time_count = 0
  for j in indices:
    time.append(dataframe.loc[j, 'time'])


  # indices記錄所有Action==act的index
  groups = [] # 用來暫存一組(同action)資料的, 型態是[][]
  temp = [] # 用來暫存一行資料的
  group_count = 0
  for i in range(len(indices)):
    if i == len(indices)-1:
      temp.append(indices[i])
      groups.append(temp)
      temp = []
      break # 如果i已經來到最後的話就break
    temp.append(indices[i])
    #time_count = time_count + 1
    if time[i+1]-16 > time[i]: #如果下個index不是連續的話, 就將前面這些存成第一組
      group_count+=1
      #time_count = time_count + 1
      groups.append(temp)
      temp = []

  #print(groups)

  fs = 64
  # window_length = 1
  # window_length = int(window_length*fs)

  final_dataframe = pd.DataFrame()
  sumOfAct=0

  for i in groups: # group[][]的每一行i
    required = math.floor(len(i)/int(window_length/2))

    
    sumOfAct= sumOfAct+required

    req_index = i[0:(required*int(window_length/2))]

    #print(req_index)
    # concat([要結合的data集合], axis=0是方向為直的)
    final_dataframe = pd.concat([final_dataframe,dataframe.iloc[req_index,:]],axis = 0)
  
  

  return final_dataframe

In [7]:
for patient in patients:

  name = first_processing_path + "/win_"+ patient +".csv"

  if os.path.isfile(name):
    dataset = pd.read_csv(name)

    activities = []
    # print(patient)
    for act in range(3):
      activities.append(create_window(act,window_length,dataset))
    to_write = pd.concat(activities,axis = 0)
    to_path = second_processing_path + "/windowed_" + patient + ".csv"
    to_write.to_csv(to_path,index = False)

del name, to_path, to_write, dataset, activities, patient, act

In [8]:
## 將 train test 以 patients 區分

train_patients_dataset = pd.DataFrame()
test_patients_dataset = pd.DataFrame()

for patient in patients:
    name = second_processing_path + "\\windowed_" + patient + ".csv"

    if not os.path.isfile(name):
        continue
    
    dataset = pd.read_csv(name)

    if patient == test_patients[0] or patient == test_patients[1]:
        test_patients_dataset = test_patients_dataset.append(dataset)
    else:    
        train_patients_dataset = train_patients_dataset.append(dataset)

to_path = final_dataset_path + "/not_windowed_train.csv"
train_patients_dataset.to_csv(to_path, index=False)

to_path = final_dataset_path + "/not_windowed_test.csv"
test_patients_dataset.to_csv(to_path, index=False)

del name, dataset, patient, test_patients_dataset, train_patients_dataset

In [9]:
data_path = os.getcwd() + "/dataset/not_windowed_train.csv"
dataset = pd.read_csv(data_path)

import math

activities = []
for act in range(3):
    activities.append(create_window(act,window_length,dataset))
to_write = pd.concat(activities, axis = 0)
to_path = os.getcwd() + "/dataset/total_train.csv"
to_write.to_csv(to_path)

In [10]:
data_path = os.getcwd() + "/dataset/not_windowed_test.csv"
dataset = pd.read_csv(data_path)

import math

activities = []
for act in range(3):
    activities.append(create_window(act,window_length,dataset))
to_write = pd.concat(activities, axis = 0)
to_path = os.getcwd() + "/dataset/total_test.csv"
to_write.to_csv(to_path)